In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.datasets import make_regression, make_swiss_roll
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import SVR

In [2]:
df = pd.read_csv("tedsd_puf_2019.csv")
df.head()

,DISYR,CASEID,STFIPS,CBSA2010,EDUC,MARSTAT,SERVICES,DETCRIM,LOS,PSOURCE,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,2019,20191553576,2,-9,4,1,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,1
1,2019,20191465214,2,-9,3,1,7,-9,35,1,...,0,0,0,0,0,0,9,4,0,3
2,2019,20191443889,2,-9,2,1,7,-9,35,1,...,0,0,0,0,0,0,9,4,0,3
3,2019,20191409377,2,-9,3,1,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,3
4,2019,20191479567,2,-9,3,3,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,1


In [3]:
df1 = df.copy()
# from conversation with "house of Hope recovery" class 4 which is "transfer to a different facility" would also be considered a success.
for value in df1['REASON']:
    if value == 1:
        value = value
    elif value == 4:
        value = 1
    else:
        value = 0
df1

,DISYR,CASEID,STFIPS,CBSA2010,EDUC,MARSTAT,SERVICES,DETCRIM,LOS,PSOURCE,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,2019,20191553576,2,-9,4,1,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,1
1,2019,20191465214,2,-9,3,1,7,-9,35,1,...,0,0,0,0,0,0,9,4,0,3
2,2019,20191443889,2,-9,2,1,7,-9,35,1,...,0,0,0,0,0,0,9,4,0,3
3,2019,20191409377,2,-9,3,1,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,3
4,2019,20191479567,2,-9,3,3,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722498,2019,20191743528,56,-9,4,2,7,-9,33,1,...,0,0,0,0,0,0,8,4,0,1
1722499,2019,20191666713,56,-9,3,3,7,1,13,7,...,0,0,0,0,0,0,8,4,0,2
1722500,2019,20191405666,56,-9,2,1,6,-9,33,1,...,0,0,0,0,0,0,8,4,0,2
1722501,2019,20191697509,56,-9,1,1,7,3,14,7,...,0,0,0,0,0,0,8,4,0,3


In [ ]:
# time to clean up the data starting with SUB1_D 
# alcohol could be a success, mmj could be a success

for value in df1['SUB1_D']:
    if value == 1 or value == 14 or value == 16 or value == 18 or value == 12 or value == 13 or value == 2:
        value = 1
    else:
        value = 0
df1

In [ ]:
# time to clean up the data starting with FREQ1_D 
# alcohol could be a success, mmj could be a success

for value in df1['SUB1_D']:
    if value == 1:
        value = 1
    else:
        value = 0
df1

In [ ]:
# time to clean up the data starting with ARRESTS and ARRESTS_D 
# alcohol could be a success, mmj could be a success

for value in df1['ARRESTS']:
    if value == -9:
        value = 0
    else:
        value = value
for value in df1['ARRESTS_D']:
    if value == -9:
        value = 0
    else:
        value = value
df1

In [4]:
reason = df1['REASON']
df1_corr = df1.corr()
df1_corr.unstack().sort_values()
variable = df1_corr['REASON'].sort_values()
variable.tail(20)

MARSTAT                0.138023
FRSTUSE1               0.139035
VET                    0.144566
SERVICES_D             0.152773
SERVICES               0.154441
IDU                    0.155722
ROUTE1                 0.165564
EMPLOY                 0.177870
FREQ_ATND_SELF_HELP    0.183493
SUB1                   0.191675
EDUC                   0.193754
LIVARAG_D              0.206890
EMPLOY_D               0.208258
LIVARAG                0.209302
ARRESTS_D              0.209523
ARRESTS                0.209631
FREQ1_D                0.214312
SUB1_D                 0.243122
REASON                 1.000000
DISYR                       NaN
Name: REASON, dtype: float64

In [5]:
# new dataframe with only the top correlated farctors left
trimmed = df1[['SUB1_D', 'FREQ1_D', 'ARRESTS', 'ARRESTS_D', 'LIVARAG', 'EMPLOY_D', 'LIVARAG_D', 'EDUC', 'SUB1', 'FREQ_ATND_SELF_HELP', 'EMPLOY', 'ROUTE1', 'IDU', 'SERVICES', 'SERVICES_D', 'VET', 'FRSTUSE1', 'MARSTAT', 'REASON']]

In [6]:
y = trimmed["REASON"]
X = trimmed.copy()
X = X.drop(columns="REASON")
X = pd.DataFrame(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
data = [X_train_scaled, X_test_scaled, y_train, y_test]

In [7]:
def test_model(model, data):
    X_train_scaled, X_test_scaled, y_train, y_test = data
    reg = model.fit(X_train_scaled, y_train)
    print(f'Model: {type(reg).__name__}')
    print(f'Train score: {reg.score(X_train_scaled, y_train)}')
    print(f'Test Score: {reg.score(X_test_scaled, y_test)}\n')
    plt.show()    

In [ ]:
test_model(LinearRegression(), data)
test_model(KNeighborsRegressor(), data)
test_model(RandomForestRegressor(), data)
test_model(ExtraTreesRegressor(), data)
test_model(AdaBoostRegressor(), data)
test_model(SVR(C=1.0, epsilon=0.2), data)

In [8]:
variable.head(20)

HLTHINS    -0.106469
PRIMINC    -0.098816
LOS        -0.084553
ALCFLG     -0.078218
PRIMPAY    -0.059634
AGE        -0.051736
DETNLF     -0.051458
CASEID     -0.042650
DIVISION   -0.030955
DAYWAIT    -0.014194
DETCRIM    -0.006041
BARBFLG    -0.003555
DSMCRIT    -0.002691
METHUSE    -0.001831
TRNQFLG     0.002683
INHFLG      0.003850
OTCFLG      0.005075
PCPFLG      0.006215
HALLFLG     0.007157
REGION      0.009529
Name: REASON, dtype: float64

In [9]:
y = df1["REASON"]
X = df1.copy()
X = X.drop(columns="REASON")
X = pd.DataFrame(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
data = [X_train_scaled, X_test_scaled, y_train, y_test]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train_scaled, y_train)
train = (f"Training Data Score: {logistic_regression_model.score(X_train_scaled, y_train)}")
test = (f"Testing Data Score: {logistic_regression_model.score(X_test_scaled, y_test)}")
testing_predictions = logistic_regression_model.predict(X_test)
print(accuracy_score(y_test, testing_predictions))